# 3. Modeling and ML Pipeline

In [2]:
# Importar bibliotecas necessárias
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Definir parâmetros de entrada
img_height, img_width = 128, 128
batch_size = 32
data_dir = "../cherry-leaves"  # Diretório com as subpastas 'healthy' e 'powdery_mildew'

# Configurar ImageDataGenerator com data augmentation e divisão de treino/validação
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 80% treino, 20% validação
)

# Criar gerador para o conjunto de treino
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',  # Classificação binária: saudável x míldio
    subset='training',
    shuffle=True
)

# Criar gerador para o conjunto de validação
validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation',
    shuffle=False
)

# Definir a arquitetura da rede neural convolucional (CNN)
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Saída para classificação binária
])

# Compilar o modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Exibir um resumo da arquitetura
model.summary()

# Treinar o modelo
history = model.fit(
    train_generator,
    epochs=10,  # Ajuste o número de épocas conforme necessário
    validation_data=validation_generator
)

# Salvar o modelo treinado para uso posterior no dashboard
model.save("../src/model.h5")


Found 3368 images belonging to 2 classes.
Found 840 images belonging to 2 classes.


/home/codeany/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,304,769 (12.61 MB)

 Trainable params: 3,304,769 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

/home/codeany/.local/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 53s 488ms/step - accuracy: 0.7148 - loss: 0.5349 - val_accuracy: 0.9226 - val_loss: 0.2285
Epoch 2/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 48s 452ms/step - accuracy: 0.9766 - loss: 0.0836 - val_accuracy: 0.9869 - val_loss: 0.0450
Epoch 3/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 46s 438ms/step - accuracy: 0.9872 - loss: 0.0424 - val_accuracy: 0.9845 - val_loss: 0.0543
Epoch 4/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 47s 442ms/step - accuracy: 0.9879 - loss: 0.0421 - val_accuracy: 0.9929 - val_loss: 0.0340
Epoch 5/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 47s 442ms/step - accuracy: 0.9892 - loss: 0.0290 - val_accuracy: 0.9964 - val_loss: 0.0133
Epoch 6/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 46s 436ms/step - accuracy: 0.9937 - loss: 0.0252 - val_accuracy: 0.9940 - val_loss: 0.0289
Epoch 7/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 46s 436ms/step - accuracy: 0.9969 - loss: 0.0134 - val_accuracy: 0.9940 - val_loss: 0.0212
Epoch 8/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 46s 435ms/step - accuracy: 0.9817 - loss: 0

# Definir parâmetros de entrada

In [1]:
img_height, img_width = 128, 128
batch_size = 32


# Criar um gerador de dados com aumento de imagens (data augmentation)

In [2]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    "../data/cherry_leaves/",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    "../data/cherry_leaves/",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

NameError: name 'ImageDataGenerator' is not defined

# Definir a arquitetura do modelo

In [3]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Para classificação binária
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

NameError: name 'Sequential' is not defined

# Iniciar treinamento (exemplo básico)

In [4]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

NameError: name 'model' is not defined

# Salvar o modelo para uso posterior

In [5]:
model.save("../src/model.h5")

NameError: name 'model' is not defined